In [13]:
# include path to RoboCOP directory
import sys
sys.path.insert(0, '../pkg/')
from robocop_diff import nuc_diff_map, tf_diff_map, get_diff_tfs
from robocop_diff.robocop_diff_plot import plot_diff_cop

We first ran RoboCOP on 5 separate MNase-seq data sets of cadmium treatment. Here are the names of the five RoboCOP output directories.

In [2]:
dirname1 = '../../../robocop_cd_DM504_Chr1_16/'
dirname2 = '../../../robocop_cd_DM505_Chr1_16_new/'
dirname3 = '../../../robocop_cd_DM506_Chr1_16_new/'
dirname4 = '../../../robocop_cd_DM507_Chr1_16/'
dirname5 = '../../../robocop_cd_DM508_Chr1_16/'
dirnames = [dirname1, dirname2, dirname3, dirname4, dirname5]

In [5]:
# output directory for DynaCOP output
outdir = '../../../robocop_diff_cd_DM504_DM505_DM506_DM507_DM508/'

## Create nucleosome linkage map

Create nucleosome linkage map by calling ```nuc_map_multiple```. The function first calls nucleosome dyads individually from the RoboCOP outputs. It then links the nucleosomes across the outputs to form a genome-wide nucleosome map for the given sets of data. Note that the nucleosomes are linked with an assumption that there exists an order in the set out RoboCOP outputs. For example, in our case we have a time series experiment of cadmium treatment of yeast cells. In case there is no order, the dirnames need to be provided in a sequence that the users seem fit.

In [6]:
nuc_df = nuc_diff_map.nuc_map_multiple(dirnames, outdir)  

Columns of ```nuc_df```:

* **chr:** chromosome

* **dyadX:** nucleosome dyad; the dyad position in different RoboCOP outputs are denoted as dyadA, dyadB,...

* **occupancyX:** Nucleosome posterior as predicted by RoboCOP; the occupancy in the different data sets are denoted as occupancyA, occupancyB,...

* **pdyadX:** Nucleosome dyad posterior as predicted by RoboCOP; the dyaad posterior is denoted as pdyadA, pdyadB, ...

* **shift_XY:** Shift in nucleosome dyad between two consecutive RoboCOP outputs. Shift calculated as dyadY - dyadX

In [7]:
nuc_df.head()

,chr,dyadA,dyadB,occupancyA,occupancyB,pdyadA,pdyadB,shift_AB,dyadC,occupancyC,...,pdyadC,dyadD,occupancyD,shift_CD,pdyadD,dyadE,occupancyE,shift_DE,pdyadE,name
0,chrI,229.0,216.0,0.999942,0.984759,0.315144,0.182249,-13.0,230.0,0.999783,...,0.334864,222.0,0.975525,-8.0,0.163414,229.0,0.999758,7.0,0.251794,nuc_0
1,chrI,389.0,389.0,0.999969,0.996902,0.270605,0.136205,0.0,404.0,0.997403,...,0.170422,385.0,0.997682,-19.0,0.277604,381.0,0.999992,-4.0,0.275946,nuc_1
2,chrI,866.0,864.0,0.999993,0.999912,0.427427,0.518984,-2.0,864.0,0.999965,...,0.636082,864.0,0.999907,0.0,0.462330,865.0,0.999345,1.0,0.277971,nuc_2
3,chrI,1179.0,NaN,0.889191,NaN,0.272899,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nuc_3
4,chrI,1582.0,1588.0,0.999886,0.999997,0.310744,0.247687,6.0,1580.0,0.998400,...,0.277184,1582.0,0.999767,2.0,0.149365,1580.0,0.997392,-2.0,0.153465,nuc_4


## Create TF difference table

Get TF binding sites that bind with probability > 0.1 in at least one of the RoboCOP outputs.

In [10]:
tf_sites = get_diff_tfs.get_tf_diff(dirnames, outdir) 

Columns of ```tf_sites```:
    
* **chr:** chromosome
        
* **start:** start of TF binding site
        
* **end**: end of TF binding site
        
* **scoreX:** posterior of TF binding at predicted by RoboCOP
        
* **TF:** name of TF

In [11]:
tf_sites.head()

,chr,start,end,scoreA,scoreB,scoreC,scoreD,scoreE,TF
0,chrXII,460398.0,460405.0,1.0,0.0,0.0,0.0,0.0,YRM1
1,chrXII,460370.0,460377.0,1.0,0.0,0.0,0.0,0.0,YRM1
2,chrXII,459967.0,459974.0,1.0,0.0,0.0,0.0,0.0,YRM1
3,chrXII,459860.0,459867.0,1.0,0.0,0.0,0.0,0.0,YRM1
4,chrXII,460384.0,460391.0,1.0,0.0,0.0,0.0,0.0,YRM1


# Plot DynaCOP output for a segment

In [15]:
chrm = 'chrVI'
start = 212000                                                                                                                                                                         
end = 213800                                                                                                                                                                         
plot_diff_cop(outdir, dirnames, chrm, start, end, filename='MET10.pdf')

NoSectionError: No section: 'main'